<a href="https://colab.research.google.com/github/ronie-casaclang/dfd-model/blob/main/dfd_20250909.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Libraries

1.   Open CV
2.   Pillow
3.   TensorFlow
4.   MatplotLib
5.   Scikit-Learn

In [ ]:
!pip install mtcnn opencv-python pillow tensorflow matplotlib scikit-learn

## Import Libraries

In [ ]:
import os, cv2
from mtcnn import MTCNN
from PIL import Image
import numpy as np

## Load Dataset from PC Directory
(Video → Cropped Face Frames 299x299)

In [ ]:
# Paths (adjust for your machine)
input_real = "C:/Users/Ronie Casaclang/Desktop/dataset/real/"
input_fake = "C:/Users/Ronie Casaclang/Desktop/dataset/fake/"
output_real = "C:/Users/Ronie Casaclang/Desktop/frames/real/"
output_fake = "C:/Users/Ronie Casaclang/Desktop/frames/fake/"

os.makedirs(output_real, exist_ok=True)
os.makedirs(output_fake, exist_ok=True)

# Initialize detector
detector = MTCNN()
fps_extract = 5

def extract_and_crop_faces(input_path, output_path, fps_extract=5):
    for file in os.listdir(input_path):
        if file.endswith(".mp4"):
            vidcap = cv2.VideoCapture(os.path.join(input_path, file))
            fps = int(vidcap.get(cv2.CAP_PROP_FPS))
            frame_interval = max(1, fps // fps_extract)

            success, frame = vidcap.read()
            count, saved = 0, 0

            while success:
                if count % frame_interval == 0:
                    faces = detector.detect_faces(frame)
                    for i, face in enumerate(faces):
                        x, y, w, h = face['box']
                        x, y = max(0, x), max(0, y)
                        cropped_face = frame[y:y+h, x:x+w]

                        if cropped_face.size != 0:
                            face_img = Image.fromarray(cv2.cvtColor(cropped_face, cv2.COLOR_BGR2RGB))
                            face_img = face_img.resize((299, 299))
                            frame_name = f"{os.path.splitext(file)[0]}_{saved}_{i}.jpg"
                            face_img.save(os.path.join(output_path, frame_name))
                            saved += 1

                success, frame = vidcap.read()
                count += 1

            if saved > 0:
                print(f"✅ {file} → {saved} faces saved")
            else:
                print(f"⚠️ {file} → No faces detected, skipped!")

# Run for real and fake datasets
extract_and_crop_faces(input_real, output_real, fps_extract)
extract_and_crop_faces(input_fake, output_fake, fps_extract)


## Preprocess & Split



1.   Load images from frames/real/ and frames/fake/.
2.   Resize & normalize them (you already extracted as 299×299, but we’ll double-check).
3.   Label them (0 = real, 1 = fake).
4.   Split into Train / Validation / Test (70/15/15).
5.   Convert into NumPy arrays ready for training.



In [ ]:
import tensorflow as tf

# Define base path
base_dir = "C:/Users/Ronie Casaclang/Desktop/frames/"

# Common parameters
img_size = (299, 299)   # matches InceptionV3/Xception expected input
batch_size = 32

# Load datasets directly from folders
train_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir + "train",
    labels="inferred",         # auto-detect "real" and "fake"
    label_mode="categorical",  # one-hot encoding for multi-class (2 classes)
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True,
    seed=42
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir + "val",
    labels="inferred",
    label_mode="categorical",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    base_dir + "test",
    labels="inferred",
    label_mode="categorical",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=False
)

# Optimize performance with caching + prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

# Check class names (should print ["fake", "real"] or vice versa)
print("Class names:", train_ds.class_names)


What this does

*   Reads images from your train/, val/, and test/ folders.
*   Automatically labels images based on subfolder name (real = class 0, fake = class 1, or vice versa).
*   Resizes every image to 299×299 (good for Xception/InceptionV3).
*   Converts labels into one-hot vectors ([1,0] = real, [0,1] = fake).
*   Adds caching + prefetching for faster training.











## Data Augmentation

What this does
*   Data Augmentation = generating “new” training images by slightly modifying existing ones → helps the model generalize better and not just memorize.

Why it matters for Deepfake detection:
*   Faces can appear at slightly different angles, brightness, zoom, etc.
*   Augmentation teaches the model to recognize “real vs fake” even when images are not perfectly aligned.


How it connects to the workflow
*   This augmentation layer is applied only to training data (not validation/test).
*   Later in Cell 6 (Build model), you’ll put this layer at the start of your CNN model (or apply it directly on batches from train_ds).

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Define data augmentation pipeline
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),   # flip left-right
    layers.RandomRotation(0.1),        # rotate image ±10%
    layers.RandomZoom(0.1),            # zoom in/out 10%
    layers.RandomContrast(0.1),        # adjust contrast ±10%
], name="data_augmentation")

# Example: apply augmentation to one batch
for images, labels in train_ds.take(1):
    augmented_images = data_augmentation(images)
    print("Original batch shape:", images.shape)
    print("Augmented batch shape:", augmented_images.shape)


## Build Model

### What this does
*   Pretrained on ImageNet, already knows rich features (edges, textures, shapes).
*   include_top=False → remove default classifier, so we can add our own real/fake classifier.
*   trainable=False → freeze backbone first (faster training, avoids overfitting). Later you can fine-tune.

### Why it matters for Deepfake detection:
*   GlobalAveragePooling2D() → reduces feature maps into a single vector per image.
*   Dropout(0.5) → randomly drops neurons, prevents overfitting.
*   Dense(2, softmax) → final classifier with 2 outputs (real/fake).


### Loss/metrics:
*   categorical_crossentropy since labels are one-hot encoded ([1,0] vs [0,1]).
*   accuracy to track performance.

In [ ]:
from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

# Input shape (must match augmentation + dataset)
input_shape = (299, 299, 3)

# Base model: Xception pretrained on ImageNet
base_model = Xception(
    weights="imagenet",
    include_top=False,        # exclude the final classification layer
    input_shape=input_shape
)
base_model.trainable = False  # freeze weights (fine-tune later if needed)

# Build model with augmentation + base + custom classifier
inputs = layers.Input(shape=input_shape)
x = data_augmentation(inputs)        # apply augmentation from Cell 5
x = base_model(x, training=False)    # forward pass through frozen backbone
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)           # regularization
outputs = layers.Dense(2, activation="softmax")(x)  # 2 classes: real/fake

model = models.Model(inputs, outputs)

# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

# Model summary
model.summary()


## Train the Model

What this does
*   model.fit(): trains the model using batches from train_ds, validates on val_ds after each epoch.
*   model.fit(): trains the model using batches from train_ds, validates on val_ds after each epoch.
*   EarlyStopping: stops training if validation loss doesn’t improve for 5 epochs (avoids overfitting).
*   ModelCheckpoint: saves the best model (deepfake_xception_best.h5) based on validation loss.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

checkpoint = ModelCheckpoint(
    "deepfake_xception_best.h5",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

# Training
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,                 # start small (10–20), can increase if needed
    callbacks=[early_stop, checkpoint],
    verbose=1
)


## Visualize Training Results

What this does
*   Extracts accuracy and loss values from the history object (Cell 7).

Create 2 plots:
*   Accuracy plot: shows how well the model learns to classify real vs fake.
*   Loss plot: shows whether the model is converging or overfitting.

In [ ]:
import matplotlib.pyplot as plt

# Get training history
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs_range = range(1, len(acc) + 1)

# Plot Accuracy
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label="Training Accuracy")
plt.plot(epochs_range, val_acc, label="Validation Accuracy")
plt.legend(loc="lower right")
plt.title("Training vs Validation Accuracy")

# Plot Loss
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label="Training Loss")
plt.plot(epochs_range, val_loss, label="Validation Loss")
plt.legend(loc="upper right")
plt.title("Training vs Validation Loss")

plt.show()


## Evaluate on Test Data

What this does
*   Runs the trained model on the test dataset (completely unseen images).

Reports:
*   Test Accuracy → how well the model generalizes to new data.
*   Test Loss → measure of prediction error.

In [ ]:
# Evaluate the trained model on the unseen test set
test_loss, test_acc = model.evaluate(test_ds, verbose=2)

print("\n✅ Test Accuracy:", round(test_acc * 100, 2), "%")
print("✅ Test Loss:", round(test_loss, 4))


## Export Model for Django

1.   List itemSavedModel format → recommended for serving inside TensorFlow Serving or directly loading in Django.
2.   H5 format → widely compatible, you can load_model("deepfake_model.h5") easily in Django views.
3.   TFLite format → optimized for mobile/edge, useful if you later extend your project to Android/iOS apps



What this does
*   Runs the trained model on the test dataset (completely unseen images).

Reports:
*   Test Accuracy → how well the model generalizes to new data.
*   Test Loss → measure of prediction error.

In [ ]:
import os

# 1. Save the model in TensorFlow SavedModel format (good for serving)
saved_model_dir = "exported_model/saved_model"
model.save(saved_model_dir)

print("✅ Model saved in TensorFlow SavedModel format at:", saved_model_dir)

# 2. Save as HDF5 (.h5) format (optional, widely supported in Keras/Django integrations)
h5_model_path = "exported_model/deepfake_model.h5"
model.save(h5_model_path)

print("✅ Model also saved as H5 file at:", h5_model_path)

# 3. Convert to TensorFlow Lite (.tflite) for mobile/edge deployment
import tensorflow as tf

tflite_model_path = "exported_model/deepfake_model.tflite"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print("✅ Model converted to TFLite at:", tflite_model_path)
